In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

#Scrapes the table of top 250 Beers

url = "https://www.beeradvocate.com/beer/top-rated/"

data = requests.get(url)
page = BeautifulSoup(data.text)
#my_page = page.get_text()

#Gets all elements in table
table = page.find("table")
rows = list()

#Puts all the links in a list
links = []
names = []

for link in table.find_all("a",href=True):
    links.append(link['href'])
    names.append(link.get_text())
    
#Creates Dataframe with all the needed links
links = pd.DataFrame(links)
names = pd.DataFrame(names)

df_names = pd.DataFrame(names.values.reshape(250,3))
df_final = pd.DataFrame(links.values.reshape(250,3))

df_final.columns = ["Beer","Company","Beer Type"]
df_final.insert(loc=0,column="Name",value=df_names[0])
df_final

,Name,Beer,Company,Beer Type
0,Kentucky Brunch Brand Stout,/beer/profile/23222/78820/,/beer/profile/23222/,/beer/top-styles/157/
1,Vanilla Bean Assassin,/beer/profile/23222/113831/,/beer/profile/23222/,/beer/top-styles/157/
2,Marshmallow Handjee,/beer/profile/26/42349/,/beer/profile/26/,/beer/top-styles/84/
3,Abraxas - Barrel-Aged,/beer/profile/25888/87246/,/beer/profile/25888/,/beer/top-styles/157/
4,Heady Topper,/beer/profile/46317/16814/,/beer/profile/46317/,/beer/top-styles/140/
...,...,...,...,...
245,Kentuckley,/beer/profile/34258/237135/,/beer/profile/34258/,/beer/top-styles/157/
246,Schaarbeekse Kriek,/beer/profile/2216/17647/,/beer/profile/2216/,/beer/top-styles/10/
247,Red Eye November,/beer/profile/33824/201187/,/beer/profile/33824/,/beer/top-styles/84/
248,Flora,/beer/profile/22511/62290/,/beer/profile/22511/,/beer/top-styles/129/


In [2]:
#GIGA SCRAPER

f = 0
# Results = pd.DataFrame(columns=['Product','UserRating','Reviews'])

rating = []
review = []
ProductList = []
ProductList2 = []
my_reviews =[]

for j in df_final['Beer']:
    #iterates through all links
    url = "https://www.beeradvocate.com/beer/" + j

    data = requests.get(url)
    page = BeautifulSoup(data.text)

    reviews = []
    for tag in page.find_all(id="rating_fullview_container"):
        reviews.append(tag.get_text())
    reviews = pd.DataFrame(reviews)

    #Prints product name
    print(df_final["Name"][f])

    ProductList2.append(df_final["Name"][f])
    # rating = []
    # review = []

    #gets the title
    Product = str(page.select('h1'))
    Product = re.search('>(.*)<br',Product)
    #Product = Product.group()
    #print(Product)

    # ProductList = []

    strings = []

    for i in reviews: #TO RUN ALL, DELETE THE [0]
        ProductList.append(Product)
        print(Product)
        first_word = i.split()[0]
        
        #Small Easy Reviews
        if first_word == "Rated:" or first_word == "Rated":
            review.append(i)
            rating.append(i.split()[1])    
            
        #Bigger Reviews Getting overall score, HARD TO GET JUST THE NUMBER CHARACTER
        elif first_word == "Reviewed":
            #Gets overall review
            review.append(i)
            splitstring = i.split("|")[4].split(" ")[2]
            splitstring = splitstring[0:4]

            #print(splitstring[0])
            if splitstring[0] == "5":
                
                strings.append(splitstring[0])
                # add decimal if a 5 W
                print("hi")

            #print(splitstring)
            strings.append(splitstring)
            
            #Come back to this for fixing rating
            rating.append(splitstring)

        else:
            print(first_word)
            rating.append("N/A")

    Results = pd.DataFrame([Product,rating,review]).T
    f = f +1
    Results.columns = ['Product','UserRating',"Reviews"]


Kentucky Brunch Brand Stout
Vanilla Bean Assassin
Marshmallow Handjee
Abraxas - Barrel-Aged
Heady Topper
Blessed
Mornin' Delight
King Julius
King JJJuliusss
Zenne Y Frontera
Pliny The Younger
Barrel Aged Imperial German Chocolate Cupcake Stout
Fundamental Observation
Very Hazy
A Deal With The Devil - Triple Oak-Aged
Bourbon County Brand Coffee Stout
A Deal With The Devil - Double Oak-Aged
Very GGGreennn
Chemtrailmix - Rye Barrel
V.S.O.J.
Juice Machine
Anabasis
It Was All A Dream
Julius
Fou' Foune
Very Green
Assassin
Westly
Speedway Stout - Vietnamese Coffee - Bourbon-Barrel Aged
Pliny The Elder
CBS (Canadian Breakfast Stout)
Monster Tones
Rare Scooop
Abner
Double Sunshine
Framboise Du Fermier
Morning Wood
JJJuliusss!
The Adjunct Trail - Bourbon Barrel-Aged
Dinner
Duck Duck Gooze
Lou Pepe - Kriek
Samuel
Parabola
Trappist Westvleteren 12 (XII)
Double Citra®
Very HHHazyyy
Doubleganger
Maman
Ann
Fuzzy
Nectarine Premiere
KBS
Bourbon County Brand Stout
Coconut Vibes
Ghost In The Machine - Do

In [3]:
pages = ["?view=beer&show=recent&start=0#lists","?view=beer&show=recent&start=40#lists","?view=beer&show=recent&start=80#lists","?view=beer&show=recent&start=120#lists"]

url = "https://www.beeradvocate.com/beer/profile/23222/78820/"

for k in pages:
    pageofreview = url+k
    # print(pages(k))
    print(pageofreview)

https://www.beeradvocate.com/beer/profile/23222/78820/?view=beer&show=recent&start=0#lists
https://www.beeradvocate.com/beer/profile/23222/78820/?view=beer&show=recent&start=40#lists
https://www.beeradvocate.com/beer/profile/23222/78820/?view=beer&show=recent&start=80#lists
https://www.beeradvocate.com/beer/profile/23222/78820/?view=beer&show=recent&start=120#lists


In [5]:
#Going through multiple pages

pages = ["?view=beer&show=recent&start=0#lists",
"?view=beer&show=recent&start=40#lists",
"?view=beer&show=recent&start=80#lists",
"?view=beer&show=recent&start=120#lists"]

rating = []
strings = []
review = []
ProductList = []

fullreviews = []

# my_pages = []


for nextpage in pages:

    url = "https://www.beeradvocate.com/beer/profile/23222/78820/" + str(nextpage)
    #pageofreview = url+nextpage
    print(url)

    data = requests.get(url)
    beer_html = data.text
    page = BeautifulSoup(beer_html,'html.parser')
    
    # my_pages.append(page.prettify())
    #Gets each review in a dataframe
    reviews = []
    for tag in page.find_all(id="rating_fullview_container"):
        reviews.append(tag.get_text())
        fullreviews.append(tag.get_text())
    #reviews = pd.DataFrame(reviews)

    # rating = []
    # strings = []
    # review = []

#Gets Beer name adds to Product List
    Product = str(page.select('h1'))
    Product = re.search('>(.*)<br',Product)
    Product = Product.group(1)
    ProductList.append(Product)

    for i in reviews:
        ProductList.append(Product)
        first_word = i.split()[0]
        
        #Small Easy Reviews
        if first_word == "Rated:" or first_word == "Rated":
            review.append("N/A") #Makes sure that each review belongs to same user
            rating.append(i.split()[1])    
            
        #Bigger Reviews Getting overall score, HARD TO GET JUST THE NUMBER CHARACTER
        elif first_word == "Reviewed":
            review.append(i) #Gets overall review into list

            splitstring = i.split("|")[4].split(" ")[2]
            splitstring = splitstring[0:4]

            #print(splitstring[0])
            if splitstring[0] == "5":
                
                strings.append(splitstring[0])
                # add decimal if a 5 W
                # print("hi")

            #print(splitstring)
            strings.append(splitstring)
            
            rating.append(splitstring)
        else:
            print(first_word)
            rating.append("N/A")

Results = pd.DataFrame([ProductList,rating,review]).T
Results.columns = ['Product','UserRating',"Reviews"]
    #Results.to_csv(Product+" beer .csv")
Results

https://www.beeradvocate.com/beer/profile/23222/78820/?view=beer&show=recent&start=0#lists
https://www.beeradvocate.com/beer/profile/23222/78820/?view=beer&show=recent&start=40#lists
https://www.beeradvocate.com/beer/profile/23222/78820/?view=beer&show=recent&start=80#lists
https://www.beeradvocate.com/beer/profile/23222/78820/?view=beer&show=recent&start=120#lists


,Product,UserRating,Reviews
0,Kentucky Brunch Brand Stout,4.9,N/A
1,Kentucky Brunch Brand Stout,5,N/A
2,Kentucky Brunch Brand Stout,5,N/A
3,Kentucky Brunch Brand Stout,5,N/A
4,Kentucky Brunch Brand Stout,5I,Reviewed by smokinjimmyv from Illinois5/5 rDe...
...,...,...,...
159,Kentucky Brunch Brand Stout,5,N/A
160,Kentucky Brunch Brand Stout,None,None
161,Kentucky Brunch Brand Stout,None,None
162,Kentucky Brunch Brand Stout,None,None


In [6]:
#Single Review Page Scrape

#product name, product review, user rating

pages = ["?view=beer&show=recent&start=0#lists","?view=beer&show=recent&start=40#lists","?view=beer&show=recent&start=80#lists","?view=beer&show=recent&start=120#lists"]

url = "https://www.beeradvocate.com/beer/profile/23222/78820/"


# url = "https://www.beeradvocate.com/beer/profile/23222/113831/"
# url = "https://www.beeradvocate.com/beer/profile/26/42349/"
# url = "https://www.beeradvocate.com/beer/profile/25888/87246/"
# url = "https://www.beeradvocate.com/beer/profile/46317/16814/"


data = requests.get(url)
page = BeautifulSoup(data.text)

#Gets each review in a dataframe
reviews = []
for tag in page.find_all(id="rating_fullview_container"):
    reviews.append(tag.get_text())
reviews = pd.DataFrame(reviews)

rating = []
strings = []
review = []

Product = str(page.select('h1'))
Product = re.search('>(.*)<br',Product)
Product = Product.group(1)

ProductList = []
# print(reviews[0])

for i in reviews[0]:
    ProductList.append(Product)
    first_word = i.split()[0]
    
    #Small Easy Reviews
    if first_word == "Rated:" or first_word == "Rated":
        review.append("N/A") #Makes sure that each review belongs to same user
        rating.append(i.split()[1])    
        
    #Bigger Reviews Getting overall score, HARD TO GET JUST THE NUMBER CHARACTER
    elif first_word == "Reviewed":
        review.append(i) #Gets overall review into list

        splitstring = i.split("|")[4].split(" ")[2]
        splitstring = splitstring[0:4]

        #print(splitstring[0])
        if splitstring[0] == "5":
            
            strings.append(splitstring[0])
            # add decimal if a 5 W
            # print("hi")

        #print(splitstring)
        strings.append(splitstring)
        
        rating.append(splitstring)
    else:
        print(first_word)
        rating.append("N/A")
Results = pd.DataFrame([ProductList,rating,review]).T
Results.columns = ['Product','UserRating',"Reviews"]
#Results.to_csv(Product+" beer .csv")
Results

,Product,UserRating,Reviews
0,Kentucky Brunch Brand Stout,4.9,N/A
1,Kentucky Brunch Brand Stout,5,N/A
2,Kentucky Brunch Brand Stout,5,N/A
3,Kentucky Brunch Brand Stout,5,N/A
4,Kentucky Brunch Brand Stout,5I,Reviewed by smokinjimmyv from Illinois5/5 rDe...
5,Kentucky Brunch Brand Stout,4.69,N/A
6,Kentucky Brunch Brand Stout,5,N/A
7,Kentucky Brunch Brand Stout,4.74,N/A
8,Kentucky Brunch Brand Stout,4.96,N/A
9,Kentucky Brunch Brand Stout,3.75,Reviewed by Emichaelbray from Pennsylvania3.79...
